In [ ]:
import numpy as np
import pandas as pd
import random

def simulate_human_decision(risk, time_pressure, reward, mood, prev_loss):
    score = 0

    score += reward * 0.6
    score -= risk * 0.7
    score -= prev_loss * 0.8
    score -= time_pressure * 0.3
    score += (1 - mood) * 0.4  # calm mood helps

    noise = random.uniform(-0.2, 0.2)
    score += noise

    return 1 if score > 0.1 else 0

def generate_dataset(samples=5000):
    data = []

    prev_decision = 0
    prev_loss = 0

    for _ in range(samples):
        risk = np.random.rand()
        time_pressure = np.random.rand()
        reward = np.random.rand()
        mood = np.random.rand()  # 0 = calm, 1 = stressed
        reaction_time = np.random.uniform(0.5, 4.0)

        decision = simulate_human_decision(
            risk, time_pressure, reward, mood, prev_loss
        )

        prev_loss = 1 if decision == 0 and reward > 0.6 else 0

        data.append([
            risk, time_pressure, reward, mood,
            prev_loss, reaction_time, decision
        ])

    columns = [
        "risk", "time_pressure", "reward",
        "mood", "previous_loss",
        "reaction_time", "decision"
    ]

    df = pd.DataFrame(data, columns=columns)
    df.to_csv("neurotwin_data.csv", index=False)
    print("Dataset generated successfully!")

generate_dataset()


Dataset generated successfully!


In [ ]:
import torch
import torch.nn as nn

class NeuroTwin(nn.Module):
    def __init__(self, input_size=6, hidden_size=64):
        super(NeuroTwin, self).__init__()

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            batch_first=True
        )

        self.decision_head = nn.Linear(hidden_size, 1)
        self.confidence_head = nn.Linear(hidden_size, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_state = lstm_out[:, -1, :]

        decision = self.sigmoid(self.decision_head(last_state))
        confidence = self.sigmoid(self.confidence_head(last_state))

        return decision, confidence


In [ ]:
import torch
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv("neurotwin_data.csv")

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Convert to sequences
sequence_length = 5
X_seq_list, y_seq_list = [], []

for i in range(len(X) - sequence_length):
    X_seq_list.append(X[i:i+sequence_length])
    y_seq_list.append(y[i+sequence_length])

# Convert lists to numpy arrays first to avoid UserWarning and improve performance
X_seq = torch.tensor(np.array(X_seq_list), dtype=torch.float32)
y_seq = torch.tensor(np.array(y_seq_list), dtype=torch.float32).unsqueeze(1)

model = NeuroTwin()
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(20):
    optimizer.zero_grad()

    decision, confidence = model(X_seq)
    loss = criterion(decision, y_seq)

    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

torch.save(model.state_dict(), "neurotwin_model.pth")
print("Model trained & saved!")

Epoch 1, Loss: 0.7096
Epoch 2, Loss: 0.7030
Epoch 3, Loss: 0.6967
Epoch 4, Loss: 0.6907
Epoch 5, Loss: 0.6849
Epoch 6, Loss: 0.6792
Epoch 7, Loss: 0.6738
Epoch 8, Loss: 0.6685
Epoch 9, Loss: 0.6634
Epoch 10, Loss: 0.6584
Epoch 11, Loss: 0.6535
Epoch 12, Loss: 0.6487
Epoch 13, Loss: 0.6441
Epoch 14, Loss: 0.6396
Epoch 15, Loss: 0.6352
Epoch 16, Loss: 0.6311
Epoch 17, Loss: 0.6271
Epoch 18, Loss: 0.6234
Epoch 19, Loss: 0.6201
Epoch 20, Loss: 0.6172
Model trained & saved!


In [ ]:
import torch
import numpy as np

model = NeuroTwin()
model.load_state_dict(torch.load("neurotwin_model.pth"))
model.eval()

# Sample scenario
sample = np.array([
    [0.8, 0.2, 0.9, 0.3, 0, 1.2]
] * 5)

sample = torch.tensor(sample, dtype=torch.float32).unsqueeze(0)

decision, confidence = model(sample)

print("Predicted Decision:", "ACCEPT" if decision.item() > 0.5 else "REJECT")
print("Confidence:", round(confidence.item(), 2))

Predicted Decision: REJECT
Confidence: 0.54


In [ ]:
def explain_decision(input_data):
    explanations = []

    if input_data["risk"] > 0.7:
        explanations.append("High risk reduced acceptance")

    if input_data["reward"] > 0.7:
        explanations.append("High reward increased motivation")

    if input_data["mood"] > 0.6:
        explanations.append("Stress affected confidence")

    if input_data["previous_loss"] == 1:
        explanations.append("Past loss caused hesitation")

    return explanations


In [ ]:
import gradio as gr
import torch
import numpy as np

# Load trained model
model = NeuroTwin()
model.load_state_dict(torch.load("neurotwin_model.pth", map_location="cpu"))
model.eval()

# Explanation logic (human-readable)
def explain_decision(risk, time_pressure, reward, mood, prev_loss):
    reasons = []

    if risk > 0.7:
        reasons.append("High risk reduced willingness to accept")

    if reward > 0.7:
        reasons.append("High reward increased motivation")

    if time_pressure > 0.6:
        reasons.append("Time pressure increased stress")

    if mood > 0.6:
        reasons.append("Stressed mood lowered confidence")

    if prev_loss == 1:
        reasons.append("Past loss caused hesitation")

    if not reasons:
        reasons.append("Balanced conditions led to a neutral decision")

    return " • " + "\n • ".join(reasons)

# Prediction function
def neurotwin_predict(risk, time_pressure, reward, mood, prev_loss, reaction_time):
    # Create 5-step decision memory (LSTM sequence)
    sample = np.array([
        [risk, time_pressure, reward, mood, prev_loss, reaction_time]
    ] * 5)

    sample = torch.tensor(sample, dtype=torch.float32).unsqueeze(0)

    with torch.no_grad():
        decision, confidence = model(sample)

    decision_label = "ACCEPT ✅" if decision.item() > 0.5 else "REJECT ❌"
    confidence_score = round(confidence.item() * 100, 2)

    explanation = explain_decision(
        risk, time_pressure, reward, mood, prev_loss
    )

    return decision_label, f"{confidence_score}%", explanation

# Gradio UI
with gr.Blocks(title="NeuroTwin AI – Human Decision Digital Twin") as demo:
    gr.Markdown("""
    # 🧠 NeuroTwin AI
    ### Artificial Neural Digital Twin of Human Decision-Making

    This system learns **how a human decides**, not just *what* they decide.
    """)

    with gr.Row():
        with gr.Column():
            risk = gr.Slider(0, 1, value=0.5, label="Risk Level")
            time_pressure = gr.Slider(0, 1, value=0.5, label="Time Pressure")
            reward = gr.Slider(0, 1, value=0.5, label="Reward Value")
            mood = gr.Slider(0, 1, value=0.5, label="Stress Level (Mood)")
            prev_loss = gr.Radio([0, 1], label="Previous Loss", value=0)
            reaction_time = gr.Slider(0.5, 4.0, value=1.5, label="Reaction Time (seconds)")

            predict_btn = gr.Button("🧠 Predict Decision")

        with gr.Column():
            decision_out = gr.Textbox(label="Predicted Decision")
            confidence_out = gr.Textbox(label="Confidence Level")
            explanation_out = gr.Textbox(
                label="Decision Explanation",
                lines=6
            )

    predict_btn.click(
        neurotwin_predict,
        inputs=[
            risk, time_pressure, reward,
            mood, prev_loss, reaction_time
        ],
        outputs=[
            decision_out,
            confidence_out,
            explanation_out
        ]
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7d5a5c5822b569db1b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import torch
import os
import numpy as np

MODEL_DIR = "user_profiles"
BASE_MODEL_PATH = "neurotwin_model.pth"

os.makedirs(MODEL_DIR, exist_ok=True)

# Load or create user-specific model
def load_user_model(username):
    model = NeuroTwin()
    user_model_path = f"{MODEL_DIR}/{username}.pth"

    if os.path.exists(user_model_path):
        model.load_state_dict(torch.load(user_model_path, map_location="cpu"))
    else:
        model.load_state_dict(torch.load(BASE_MODEL_PATH, map_location="cpu"))
        torch.save(model.state_dict(), user_model_path)

    model.eval()
    return model, user_model_path

# Explanation engine
def explain(risk, reward, mood, prev_loss):
    reasons = []
    if risk > 0.7:
        reasons.append("High risk caused hesitation")
    if reward > 0.7:
        reasons.append("High reward encouraged acceptance")
    if mood > 0.6:
        reasons.append("Stress reduced confidence")
    if prev_loss == 1:
        reasons.append("Past loss influenced decision")

    return "\n".join(reasons) if reasons else "Balanced conditions"

# Prediction function
def predict(username, risk, time_pressure, reward, mood, prev_loss, reaction_time):
    if not username:
        return "❌ Enter username", "", ""

    model, _ = load_user_model(username)

    seq = np.array([
        [risk, time_pressure, reward, mood, prev_loss, reaction_time]
    ] * 5)

    seq = torch.tensor(seq, dtype=torch.float32).unsqueeze(0)

    with torch.no_grad():
        decision, confidence = model(seq)

    label = "ACCEPT ✅" if decision.item() > 0.5 else "REJECT ❌"
    conf = f"{round(confidence.item()*100,2)}%"
    explanation = explain(risk, reward, mood, prev_loss)

    return label, conf, explanation

# Online learning (user brain evolves)
def train_user(username, risk, time_pressure, reward, mood, prev_loss, reaction_time, true_decision):
    model, path = load_user_model(username)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
    loss_fn = torch.nn.BCELoss()

    seq = np.array([
        [risk, time_pressure, reward, mood, prev_loss, reaction_time]
    ] * 5)

    seq = torch.tensor(seq, dtype=torch.float32).unsqueeze(0)
    target = torch.tensor([[true_decision]], dtype=torch.float32)

    decision, _ = model(seq)
    loss = loss_fn(decision, target)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    torch.save(model.state_dict(), path)
    return "✅ User cognitive profile updated!"

# Gradio UI
with gr.Blocks(title="NeuroTwin AI – Multi-User Cognitive Profiles") as demo:
    gr.Markdown("""
    # 🧠 NeuroTwin AI
    ## Multi-User Cognitive Digital Twin System

    Each user has a **personal neural brain** that learns how *they* decide.
    """)

    username = gr.Textbox(label="👤 Username (New or Existing)")

    with gr.Row():
        with gr.Column():
            risk = gr.Slider(0,1,0.5,label="Risk Level")
            time_pressure = gr.Slider(0,1,0.5,label="Time Pressure")
            reward = gr.Slider(0,1,0.5,label="Reward Value")
            mood = gr.Slider(0,1,0.5,label="Stress Level")
            prev_loss = gr.Radio([0,1],label="Previous Loss",value=0)
            reaction_time = gr.Slider(0.5,4.0,1.5,label="Reaction Time")

        with gr.Column():
            decision_out = gr.Textbox(label="Predicted Decision")
            confidence_out = gr.Textbox(label="Confidence")
            explanation_out = gr.Textbox(label="Explanation", lines=6)

    predict_btn = gr.Button("🧠 Predict Decision")
    predict_btn.click(
        predict,
        inputs=[
            username, risk, time_pressure, reward,
            mood, prev_loss, reaction_time
        ],
        outputs=[decision_out, confidence_out, explanation_out]
    )

    gr.Markdown("### 🧬 Train User Brain (Online Learning)")
    true_decision = gr.Radio([0,1], label="Actual User Decision (0=Reject, 1=Accept)")
    train_btn = gr.Button("🔁 Update Cognitive Profile")

    train_btn.click(
        train_user,
        inputs=[
            username, risk, time_pressure, reward,
            mood, prev_loss, reaction_time, true_decision
        ],
        outputs=gr.Textbox()
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3557cf2a16623be82a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import torch
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

def extract_brain_vector(model):
    """Flatten neural weights into 1 vector"""
    weights = []
    for param in model.parameters():
        weights.append(param.detach().cpu().numpy().flatten())
    return np.concatenate(weights)

# 🧠 Brain Drift
def brain_drift(old_vec, new_vec):
    return np.linalg.norm(new_vec - old_vec)

# 🧠 Cognitive Similarity
def cognitive_similarity(vec1, vec2):
    return cosine_similarity([vec1], [vec2])[0][0]

# 🧠 Personality Clustering
def cluster_users(brain_vectors, n_clusters=3):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(brain_vectors)
    return labels


In [ ]:
import torch
import torch.nn as nn

class NeuroTwin(nn.Module):
    def __init__(self, input_size=6, hidden_size=64):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.decision = nn.Linear(hidden_size, 1)
        self.confidence = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.lstm(x)
        h = out[:, -1, :]
        return self.sigmoid(self.decision(h)), self.sigmoid(self.confidence(h))


In [ ]:
import torch

model = NeuroTwin()
torch.save(model.state_dict(), "neurotwin_model.pth")
print("✅ Base model created successfully")

✅ Base model created successfully


In [ ]:
import matplotlib
matplotlib.use("Agg")

import gradio as gr
import torch, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

# ================= CONFIG =================
BASE_MODEL = "neurotwin_model.pth"
USER_DIR = "user_profiles"
os.makedirs(USER_DIR, exist_ok=True)

# ================= STATE =================
def init_state():
    return pd.DataFrame(columns=[
        "user_id","name","age",
        "decision","confidence",
        "cognitive_score","personality","timestamp"
    ])

# ================= MODEL UTILS =================
def load_user(username):
    model = NeuroTwin()
    path = f"{USER_DIR}/{username}.pth"

    if os.path.exists(path):
        model.load_state_dict(torch.load(path, map_location="cpu"))
    else:
        model.load_state_dict(torch.load(BASE_MODEL, map_location="cpu"))
        torch.save(model.state_dict(), path)

    return model, path

def model_vector(model):
    return np.concatenate([p.detach().numpy().flatten() for p in model.parameters()])

# ================= PREDICT + TABLE UPDATE =================
def predict_and_update(name, age, df,
                       risk, time_p, reward, mood, prev_loss, rt):

    if not name.strip():
        return df, df, "❌ Name required", "", "", ""

    model,_ = load_user(name)
    model.eval()

    seq = torch.tensor([[[risk,time_p,reward,mood,prev_loss,rt]]*5],
                       dtype=torch.float32)

    with torch.no_grad():
        decision_prob, confidence = model(seq)

    decision = "ACCEPT" if decision_prob.item() > 0.5 else "REJECT"

    # Cognitive score logic
    base_score = 0.6
    cognitive_score = (
        base_score + confidence.item()*0.3
        if decision == "ACCEPT"
        else base_score - confidence.item()*0.3
    )
    cognitive_score = round(float(np.clip(cognitive_score,0,1)),3)

    personality = (
        "INTJ" if decision=="ACCEPT" and risk<0.5 else
        "ENFP" if decision=="ACCEPT" else
        "ISTJ"
    )

    entry = {
        "user_id": f"U{len(df)+1:03d}",
        "name": name,
        "age": age,
        "decision": decision,
        "confidence": round(confidence.item()*100,2),
        "cognitive_score": cognitive_score,
        "personality": personality,
        "timestamp": datetime.now().strftime("%H:%M:%S")
    }

    df = pd.concat([df, pd.DataFrame([entry])], ignore_index=True)

    return (
        df,
        df,
        "✅ Decision stored in table",
        decision,
        f"{entry['confidence']}%",
        personality
    )

# ================= BRAIN DRIFT =================
def brain_drift(username):
    model,_ = load_user(username)
    vec = model_vector(model)

    hist_path = f"{USER_DIR}/{username}_history.npy"
    history = np.load(hist_path) if os.path.exists(hist_path) else np.array([vec])

    drift=[np.linalg.norm(vec-h) for h in history]
    history=np.vstack([history,vec])
    np.save(hist_path,history)

    fig,ax=plt.subplots()
    ax.plot(drift,marker="o")
    ax.set_title("Brain Drift Over Time")
    ax.set_xlabel("Updates")
    ax.set_ylabel("Drift")
    return fig

# ================= SIMILARITY =================
def similarity_users(u1,u2):
    m1,_=load_user(u1)
    m2,_=load_user(u2)
    sim=cosine_similarity([model_vector(m1)],[model_vector(m2)])[0][0]
    return f"Similarity: {round(sim*100,2)}%"

# ================= CLUSTERING =================
def cluster_users(df):
    if len(df)<2:
        return "Need ≥2 records"

    X=df[["cognitive_score"]]
    labels=KMeans(n_clusters=2,n_init=10).fit_predict(X)

    out=df.copy()
    out["cluster"]=labels
    return out[["name","decision","cluster"]]

# ================= UI =================
with gr.Blocks(title="🧠 NeuroTwin Cognitive System") as demo:

    gr.Markdown("# 🧠 NeuroTwin – Decision-Driven Cognitive Twin")

    state_df = gr.State(init_state())

    with gr.Row():
        name = gr.Textbox(label="Name")
        age = gr.Number(label="Age",value=20)

    gr.Markdown("## 🧠 Decision Inputs")
    risk=gr.Slider(0,1,0.5,label="Risk")
    time_p=gr.Slider(0,1,0.5,label="Time Pressure")
    reward=gr.Slider(0,1,0.5,label="Reward")
    mood=gr.Slider(0,1,0.5,label="Stress")
    prev_loss=gr.Radio([0,1],value=0,label="Previous Loss")
    rt=gr.Slider(0.5,4,1.5,label="Reaction Time")

    decision=gr.Textbox(label="Decision")
    confidence=gr.Textbox(label="Confidence")
    personality=gr.Textbox(label="Personality")

    status=gr.Textbox(label="Status",interactive=False)
    table=gr.Dataframe(interactive=False)

    gr.Button("🧠 Make Decision").click(
        predict_and_update,
        [name,age,state_df,risk,time_p,reward,mood,prev_loss,rt],
        [state_df,table,status,decision,confidence,personality]
    )

    gr.Markdown("## 📈 Brain Drift")
    gr.Button("Show Brain Drift").click(brain_drift,name,gr.Plot())

    gr.Markdown("## 🔁 Cognitive Similarity")
    gr.Button("Compare With Self").click(similarity_users,[name,name],gr.Textbox())

    gr.Markdown("## 🧩 Personality Clustering")
    gr.Button("Cluster Records").click(cluster_users,state_df,gr.Dataframe())

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4a6fc66dfe100e039e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
